In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

__Vasicek Model:__ $dr_t = \alpha(b-r_t)dt + \sigma dW_t$   
Точное решение: $r_t \sim N\left(r_0 e^{-\alpha t} + b (1-e^{-\alpha t}),\,\tfrac{\sigma^2}{2\alpha}\left(1-e^{-2\alpha t}\right)\right)
$

In [2]:
def minimizeNLogLikelihood(r):
    dt = 1 
    rt = r    
    N = r.shape[0]
    params0 = np.array([0.0000001,0.0000001,0.0000001])
    
    def VasicekLogLikelihood(params):
        alpha, b, sigma = params[0], params[1], params[2] 
        return -(N-1)*np.log(alpha/(sigma**2*(1-np.exp(-2*alpha*dt)))) \
            - alpha/(sigma**2*(1-np.exp(-2*alpha*dt)))*np.sum(-(rt[:-1] - rt[1:]*np.exp(-alpha*dt)-b*(1-np.exp(-alpha*dt)))**2) 
    
    params_opt = minimize(VasicekLogLikelihood, params0, method='Powell')
    return params_opt.x

In [3]:
def ClosestTradeDate(day, dates):
    return dates[dates <= day][-1]
    

In [4]:
def estimate_params(components):
    params = np.zeros((3,3))

    for i in range(3):
        params[i] = minimizeNLogLikelihood(components[:,i])

    return params

In [5]:
def load_bonds(bond_count, forDate):
 
    bond_info = {}

    for i in range( bond_count):
        bond_i = i

        coupons =pd.read_excel('офз_график_выплат.xlsx', bond_i)[['Date', 'Coupon']].astype({'Date':'M8[D]'})

        coupons_date = coupons['Date'].values



        info = pd.read_excel('офз_описание.xlsx',bond_i, index_col=0)

        coupons_rate = coupons['Coupon'].values/2/100
        coupons_rate = coupons_rate[coupons_date > forDate]

        coupons_date = coupons_date[coupons_date > forDate].reshape(-1,1)
        expDate = np.datetime64(info.loc['Дата погашения'].values[0])

#         print('bond number', i)
#         print('expDate', expDate)
#         print('coupon', coupons_rate.mean())
#         print()



        face = info.loc['Номинальная стоимость'].values[0]

        bond_info[i] = {
            'coupons_date': coupons_date,
            'coupons_rate': coupons_rate,
            'expDate': expDate,
            'face': face
        }
    return bond_info


In [6]:
def VasicekPath(params,r0, N):
    dt = 1    
    alpha, b, sigma = params[0], params[1], params[2]
    r = [] # варианты аля-ставки на один из аля-TtM  
    for j in range(N):
        r1 = r0 + alpha*(b-r0)*dt + sigma*np.sqrt(dt)*np.random.randn()
        r.append(r1)
    return r

In [7]:
from scipy.interpolate import interp1d

  
def interpolate(bond_count, pca, r0, bond_info):
    
    for bond_i in range(bond_count):
    #переводим аля-ставки в разности реальных ставок 
        inverse_r = pca.inverse_transform(bond_info[bond_i]['simulations'])

    #переводим разности в кривую YTM. 
        inverse_r = r0.values + inverse_r
    

        x = [0]+list(r0.index*360)
        xnew = np.arange(1, 30*360, 1)

        inverse_r_interpolate = np.zeros((inverse_r.shape[0], len(xnew)))

        for i, ofz in enumerate(inverse_r):
            y = [0]+list(ofz)

            f = interp1d(x, y)
            ynew = f(xnew)

            inverse_r_interpolate[i] = ynew
        bond_info[bond_i]['inverse_r_interpolate'] = inverse_r_interpolate
        
    return bond_info

In [8]:
def PriceBond(bond_params,  N=10):
    coupons_date = bond_params['coupons_date']
    couponRate = bond_params['coupons_rate']
    FaseValue = bond_params['face']/10
    inverse_r_interpolate = bond_params['inverse_r_interpolate']

    TimeToMaturity = np.apply_along_axis(lambda x: int((x - tradeDate1)/np.timedelta64(1,'D')), 1, coupons_date)
    B = np.zeros((N, TimeToMaturity[-1]))
    discount_factor = ((1+inverse_r_interpolate[:, :TimeToMaturity[-1]])**np.arange(TimeToMaturity[-1]))[:, ::-1]


    for i in range(N):    
        #для каждого купона считаем его динамику цены (делим значение купона на показатель дисконтирования)
        for j,_ in enumerate(TimeToMaturity):        
            V = FaseValue*couponRate[j] if j != len(TimeToMaturity)-1 else FaseValue*(1+couponRate[j])  
            b = (V / discount_factor[i, :TimeToMaturity[j]])  
            B[i, :TimeToMaturity[j]] += b
        
    return B

In [9]:
def estimate_r(bond_count, params, r0_diff, N=100):
    for bond_i in range(bond_count):
        r = []    
        for j in range(params.shape[0]):        
            r.append(VasicekPath(params[j], r0_diff[j], N))

        bond_info[bond_i]['simulations'] = np.array(r).T
    return bond_info

In [10]:
def estimate_price(bond_count, bond_info, N):

    for i in range(bond_count):
        #print('bond', i)
        p = PriceBond(bond_info[i], N=N)

        bond_info[i]['price_path'] = p
        
        
    return bond_info

### смотрим на весь портфель из облигаций на 1 день

In [11]:
def all_price(bond_count, N, forecastDate, tradeDate1, bond_info):
    p1_predict = np.zeros((bond_count, N))
    p0 = np.array([])
    p1 = np.array([])

    for i in range(bond_count):
        bond_price = pd.read_excel('офз_данные.xlsx', i, index_col=2).iloc[:, 6:7]
        bond_price.columns=['price']


        p0 = np.append(p0, bond_price.loc[tradeDate1]['price'])
        p1 = np.append(p1, bond_price.loc[forecastDate]['price'])

        bond_result = bond_info[i]
        dates_index = pd.date_range(tradeDate1, bond_result['expDate'])[:-1]
        p_path = pd.DataFrame(bond_result['price_path'], columns=dates_index)
        p1_predict[i] = p0[-1]*(p_path[forecastDate].values/p_path[tradeDate1].values)


    return p0, p1, p1_predict

In [12]:
def pricePortfolioInitial(p_real, bond_value = 1):
    w = bond_value/p_real
    return np.sum(w*p_real), w

def pricePortfolio(price, w):
    if len(price.shape) == 2:
        return np.sum(w[:, None]*price, 0)
    else: 
        return np.sum(w*price)

In [13]:
def estimate_VaR(p0, p1, p1_predict, alpha = 0.99):
    bond_price_true_0, w = pricePortfolioInitial(p0)
    bond_price_predict = pricePortfolio(p1_predict, w)
    
    

    looses = (bond_price_true_0 - bond_price_predict)/bond_price_true_0
    VaR = np.percentile(looses, alpha*100)
    bond_price_true_1 =  pricePortfolio(p1, w)
    loss_real = (bond_price_true_0 - bond_price_true_1)/bond_price_true_0
    print(f"port0_value: {bond_price_true_0}, port1_value: {bond_price_true_1}, port1_pred_value: {bond_price_predict.mean()}")
    return VaR, loss_real, w

In [14]:
ir = pd.read_excel('офз.xlsx', index_col=0, header=1) / 100/365

dates = np.array(['-'.join(x.split('.')[::-1]) for x in ir.index]).astype('M8[D]')
ir.index = dates
ir.sort_index(inplace=True)
dates = dates[::-1][1:]

ir_diff = ir.diff().dropna()# берем разности

pca = PCA(n_components=3)
components = pca.fit_transform(ir_diff)




In [19]:
result = []
startDate0 = np.datetime64('2014-12-03')

#result = pd.DataFrame(columns=['forDate', 'VaR', 'loss'])

for forecastDate in dates[dates >= np.datetime64('2018-01-12')][:]:
    bond_count = 5
    N = 100
    startDate = startDate0 + (forecastDate - np.datetime64('2018-01-12'))
    
    tradeDate1 = forDate= ClosestTradeDate(forecastDate-1, dates)
    
    print('last traid date before forecase', tradeDate1)
    print('forecast date', forecastDate)
    bond_info = load_bonds(bond_count, tradeDate1)
    
    components_for = components[(dates <= tradeDate1) & (dates >= startDate)]
    params = estimate_params(components_for)
    
    r0_diff = components_for[-1]
    r0 = ir.loc[tradeDate1]
    
    bond_info = estimate_r(bond_count, params,  r0_diff, N=N)
    bond_info = interpolate(bond_count, pca, r0, bond_info)
    bond_info = estimate_price(bond_count, bond_info, N)
    
    p0, p1, p1_predict = all_price(bond_count, N, forecastDate, tradeDate1, bond_info)
    VaR, loss, w = estimate_VaR(p0, p1, p1_predict)
    
    print('VaR', VaR, 'loss', loss)
    print()
    result.append([forecastDate, VaR, loss])

last traid date before forecase 2018-01-11
forecast date 2018-01-12
port0_value: 5.0, port1_value: 5.011871455456897, port1_pred_value: 5.0010728431951375
VaR -0.00020956367523242747 loss -0.0023742910913794545

last traid date before forecase 2018-01-12
forecast date 2018-01-15
port0_value: 5.0, port1_value: 5.001988754688923, port1_pred_value: 5.003228726237287
VaR -0.0006303950251866066 loss -0.00039775093778455785

last traid date before forecase 2018-01-15
forecast date 2018-01-16
port0_value: 5.0, port1_value: 4.9970922442115215, port1_pred_value: 5.001086786860196
VaR -0.00021076032909634536 loss 0.0005815511576956923

last traid date before forecase 2018-01-16
forecast date 2018-01-17
port0_value: 5.0, port1_value: 4.99730941121962, port1_pred_value: 5.001035593595777
VaR -0.0002007558108787588 loss 0.000538117756076062

last traid date before forecase 2018-01-17
forecast date 2018-01-18
port0_value: 5.0, port1_value: 5.00130884548345, port1_pred_value: 5.001056212348163
VaR -0

port0_value: 5.0, port1_value: 4.997763555983614, port1_pred_value: 5.001009061555099
VaR -0.0001970449679538948 loss 0.00044728880327724594

last traid date before forecase 2018-03-13
forecast date 2018-03-14
port0_value: 5.0, port1_value: 4.994600898515454, port1_pred_value: 5.001012067357263
VaR -0.0001965997276573024 loss 0.001079820296909162

last traid date before forecase 2018-03-14
forecast date 2018-03-15
port0_value: 5.0, port1_value: 4.993473205082073, port1_pred_value: 5.001012482528151
VaR -0.00019726642859417255 loss 0.001305358983585414

last traid date before forecase 2018-03-15
forecast date 2018-03-16
port0_value: 5.0, port1_value: 5.00437185892701, port1_pred_value: 5.001003824619528
VaR -0.00019552609997288073 loss -0.0008743717854020616

last traid date before forecase 2018-03-16
forecast date 2018-03-19
port0_value: 5.0, port1_value: 4.999625595259152, port1_pred_value: 5.003045801276764
VaR -0.0005943154552779912 loss 7.488094816956447e-05

last traid date before

port0_value: 5.0, port1_value: 5.012400462106834, port1_pred_value: 5.002082940983542
VaR -0.0004081251172541638 loss -0.0024800924213668196

last traid date before forecase 2018-05-10
forecast date 2018-05-11
port0_value: 5.0, port1_value: 5.009651288147392, port1_pred_value: 5.0010493648703145
VaR -0.00020566771104725488 loss -0.0019302576294784046

last traid date before forecase 2018-05-11
forecast date 2018-05-14
port0_value: 5.0, port1_value: 4.989871620611287, port1_pred_value: 5.0031154796791215
VaR -0.000608795677783478 loss 0.0020256758777426496

last traid date before forecase 2018-05-14
forecast date 2018-05-15
port0_value: 5.0, port1_value: 4.989702876550043, port1_pred_value: 5.001045006875701
VaR -0.00020469598499760444 loss 0.0020594246899914735

last traid date before forecase 2018-05-15
forecast date 2018-05-16
port0_value: 5.0, port1_value: 5.001925159683716, port1_pred_value: 5.001080872603649
VaR -0.00021095225779447978 loss -0.00038503193674319645

last traid date

port0_value: 5.0, port1_value: 5.008010498538628, port1_pred_value: 5.001088052020192
VaR -0.00021173795035964282 loss -0.0016020997077255573

last traid date before forecase 2018-07-05
forecast date 2018-07-06
port0_value: 5.0, port1_value: 4.997484609076035, port1_pred_value: 5.001096432456189
VaR -0.0002127543800062952 loss 0.0005030781847930044

last traid date before forecase 2018-07-06
forecast date 2018-07-09
port0_value: 5.0, port1_value: 5.016702471084016, port1_pred_value: 5.003241660368245
VaR -0.0006326596010781813 loss -0.003340494216803158

last traid date before forecase 2018-07-09
forecast date 2018-07-10
port0_value: 5.0, port1_value: 5.001972395822985, port1_pred_value: 5.001085761561684
VaR -0.0002134388741673199 loss -0.00039447916459707245

last traid date before forecase 2018-07-10
forecast date 2018-07-11
port0_value: 5.0, port1_value: 5.002979109555233, port1_pred_value: 5.0010769460854565
VaR -0.00020998412854208757 loss -0.0005958219110466345

last traid date 

port0_value: 5.0, port1_value: 4.996174338874588, port1_pred_value: 5.001225147435061
VaR -0.00023966101796205487 loss 0.0007651322250824321

last traid date before forecase 2018-08-30
forecast date 2018-08-31
port0_value: 5.0, port1_value: 5.005561802129932, port1_pred_value: 5.001202757319016
VaR -0.00023536246115060953 loss -0.0011123604259864806

last traid date before forecase 2018-08-31
forecast date 2018-09-03
port0_value: 5.0, port1_value: 5.0087569529528935, port1_pred_value: 5.0037234426584565
VaR -0.0007290607889650254 loss -0.0017513905905786943

last traid date before forecase 2018-09-03
forecast date 2018-09-04
port0_value: 5.0, port1_value: 4.975046543408029, port1_pred_value: 5.001222237754404
VaR -0.0002400255047667237 loss 0.0049906913183942695

last traid date before forecase 2018-09-04
forecast date 2018-09-05
port0_value: 5.0, port1_value: 4.988227280199099, port1_pred_value: 5.001213876221701
VaR -0.0002384082085008714 loss 0.002354543960180244

last traid date be

port0_value: 5.0, port1_value: 4.994550552620591, port1_pred_value: 5.001188224565017
VaR -0.00023361730437660455 loss 0.0010898894758817335

last traid date before forecase 2018-10-25
forecast date 2018-10-26
port0_value: 5.0, port1_value: 5.00031930210623, port1_pred_value: 5.001191815377731
VaR -0.00023484434605239814 loss -6.386042124599811e-05

last traid date before forecase 2018-10-26
forecast date 2018-10-29
port0_value: 5.0, port1_value: 5.002105268283369, port1_pred_value: 5.003582541206047
VaR -0.0007039679874712608 loss -0.0004210536566738199

last traid date before forecase 2018-10-29
forecast date 2018-10-30
port0_value: 5.0, port1_value: 4.997375377458332, port1_pred_value: 5.001210603931757
VaR -0.00023811015848089313 loss 0.0005249245083335907

last traid date before forecase 2018-10-30
forecast date 2018-10-31
port0_value: 5.0, port1_value: 5.0050879445730505, port1_pred_value: 5.00119518467814
VaR -0.00023478734191681027 loss -0.0010175889146101015

last traid date b

port0_value: 5.0, port1_value: 4.992094180509955, port1_pred_value: 5.001224700480447
VaR -0.00024060315217668915 loss 0.0015811638980089526

last traid date before forecase 2018-12-21
forecast date 2018-12-24
port0_value: 5.0, port1_value: 5.005529776593071, port1_pred_value: 5.003628655341229
VaR -0.000715392154472612 loss -0.0011059553186141713

last traid date before forecase 2018-12-24
forecast date 2018-12-25
port0_value: 5.0, port1_value: 4.996292804998852, port1_pred_value: 5.001200328303937
VaR -0.00023656877615606397 loss 0.0007414390002296045

last traid date before forecase 2018-12-25
forecast date 2018-12-26
port0_value: 5.0, port1_value: 4.999420315098365, port1_pred_value: 5.001193039157377
VaR -0.00023487508690590172 loss 0.00011593698032701383

last traid date before forecase 2018-12-26
forecast date 2018-12-27
port0_value: 5.0, port1_value: 5.01366286607854, port1_pred_value: 5.0012020175515355
VaR -0.00023647486041195975 loss -0.002732573215707923

last traid date be

In [20]:
result = pd.DataFrame(result, columns=['forDate', 'VaR', 'loss'])


In [21]:
sum(result['loss'] > result['VaR'])/result.shape[0]

0.5714285714285714